# Baseball learning network

In [1]:
import numpy as np
import tensorflow as tf
import tflearn
from tqdm import tqdm_notebook
import copy
from scipy import stats

In [29]:
class GameStats(object):
    def __init__(self, homeTeamNameIndex, homeTeamScoreIndex, homeTeamStatsIndex, visitorTeamNameIndex, visitorTeamScoreIndex, visitorTeamStatsIndex, file="baseball2016.txt"):
        #parse the text file
        self.statsFile = open(file, "r")
        self.topArray = []
        self.sideArray = []  
        self.sc = np.zeros((30,30,30), np.int32) 
        self.sc[:,:,:] = -1  
        self.am = np.zeros((30,30), np.float32)
        self.gameList = []
        
        seenTeams = []
        for line in self.statsFile:
            token = line.split(',')  #tokenize the string
            tokenIndex = [homeTeamNameIndex, homeTeamScoreIndex, visitorTeamNameIndex, visitorTeamScoreIndex] + [i for i in homeTeamStatsIndex] + [i for i in visitorTeamStatsIndex]
            attributes = dict()
            
            for i in range(len(token)):
                if(i in tokenIndex):
                    attributes[i] = self.removeQuotes(token[i])
                        
            self.addScore(attributes[homeTeamNameIndex], attributes[visitorTeamNameIndex], attributes[homeTeamScoreIndex], attributes[visitorTeamScoreIndex])                
            self.addGame(attributes[homeTeamNameIndex], attributes[homeTeamScoreIndex], [attributes[i] for i in homeTeamStatsIndex], attributes[visitorTeamNameIndex], attributes[visitorTeamScoreIndex], [attributes[i] for i in visitorTeamStatsIndex])
            
            if(attributes[homeTeamNameIndex] not in seenTeams):
                seenTeams.append(attributes[homeTeamNameIndex])
            
        self.buildAvgMatrix()
        self.statsFile.close()
        #self.gameList = [bin(2**i)[2:].zfill(len(seenTeams)) if x == seenTeams[i] else x for i in range(len(seenTeams)) for x in self.gameList]
        # take the teams and convert the indexes of the order that they appeared in into the hot one format by rasiing
        # 2 to the power of them and then converting them into binary.
        seenTeamsDict = {k: v for v, k in enumerate(seenTeams)}
        temp = []
        for x in self.gameList:
            tempi = []
            for z in x:
                if(z in seenTeams):
                    tempi.append(bin(2**seenTeamsDict[z])[2:].zfill(len(seenTeamsDict)))
                else:
                    tempi.append(z)
            temp.append(tempi)
        self.gameList = temp
        
    def removeQuotes(self, string):
        if (string.startswith('"') and string.endswith('"')) or (string.startswith("'") and string.endswith("'")):
            return string[1:-1]
        return string  
    
    def addGame(self, team1, score1, stats1, team2, score2, stats2):
        self.gameList.append([team1, score1, stats1, team2, score2, stats2])
        
    #give it two teams, the scores, and it will add it to the matrix
    def addScore(self, team1, team2, score1, score2):
        '''
        for a team in top array, the index in the array corrisponds to the matrix column there located in
        for a team in side array, the index in the array corrisponds to the matrix row there located in
        '''
        #team 1 score entry
        try:
            row = self.sideArray.index(team2)    

        except:
            self.sideArray.append(team2)
            row = self.sideArray.index(team2)    

        try:
            col = self.topArray.index(team1)
        except:
            self.topArray.append(team1)
            col = self.topArray.index(team1)
        temp = self.sc[row, col]
        counter = 0
        for e in temp:
            if (e == -1):
                temp[counter] = score1
                break
            counter += 1
        self.sc[row, col] = temp
        
        #team 2 score entry
        try:
            row = self.sideArray.index(team1)    
        except:
            self.sideArray.append(team1)
            row = self.sideArray.index(team1)    
            
        try:
            col = self.topArray.index(team2)
        except:
            self.topArray.append(team2)
            col = self.topArray.index(team2)
        temp = self.sc[row, col]
        counter = 0
        for e in temp:
            if (e == -1):
                temp[counter] = score2
                break
            counter += 1
        self.sc[row, col] = temp
    
    #returns the score(s) for match up
    def getScore(self, team1, team2, gameSelect = None):
        print(team1, team2)
        try:
            score1 = self.sc[self.sideArray.index(team2), self.topArray.index(team1)]
            score2 = self.sc[self.sideArray.index(team1), self.topArray.index(team2)]
            if (gameSelect == None):
                print(team1, score1)
                print(team2, score2)
            else:
                print(team1, score1[gameSelect])
                print(team2, score2[gameSelect])
        except:
            print('Invalid input of teams')
    
    def getGameList(self):
        return copy.deepcopy(self.gameList)
    
    #constructs a matrix of the avg score in a matchup
    def buildAvgMatrix(self): 
        for col in range(len(self.sc[:,0])):   #depth
            for row in range(len(self.sc[0, :])):  #width
                tempScore = self.sc[row, col]
                avgScore = 0.0
                count = 0.0
                for j in tempScore:
                    if (j != -1):
                        avgScore += j
                        count += 1
                    else:
                        break
                try:
                    avgScore = avgScore / count
                except:
                    avgScore = -1
                self.am[row, col] = avgScore
    
    #get the value of the avg score for a match up
    def getAvgScore(self, team1, team2):
        try:
            score1 = self.am[self.sideArray.index(team2), self.topArray.index(team1)]
            score2 = self.am[self.sideArray.index(team1), self.topArray.index(team2)]
            print(team1, score1)
            print(team2, score2)        
        except:
            print('Invalid input of teams')
    
#getting rid of the strings
def removeQuotes(gameList):
    for row in gameList:
        for x in range(len(row)):
            #convert scores strings to float
            if (x == 1 or x == 4): row[x] = float(row[x])
            #convert arrays to floats
            if (x == 2 or x == 5): row[x] = list(map(float, row[x]))
    return gameList

# split the stats up into different parts (this takes a lot of time, but I think it will be worth it)
def splitGameList(gameList):
    gameList = removeQuotes(gameList)   #because i initialized gamestats above, must remove quote again

    homeTeamName = []
    homeTeamScore = []
    homeTeamStats = []

    visitingTeamName = []
    visitingTeamScore = []
    visitingTeamStats = []

    for row in gameList:
        homeTeamName.append(list(row[0]))
        homeTeamScore.append(row[1])
        homeTeamStats.append(row[2])
        visitingTeamName.append(list(row[3]))
        visitingTeamScore.append(row[4])
        visitingTeamStats.append(row[5])
    
    return homeTeamName, homeTeamScore, homeTeamStats, visitingTeamName, visitingTeamScore, visitingTeamStats

def previousAverage(gameList):
    avgStats = dict()
    np.set_printoptions(precision=4)  #If you want to remove this i recommend restarting the kernal

    newGameList = copy.deepcopy(gameList)
    delList = []
    count = 0    
    
    #add up all the stats for each team
    for i in range(len(newGameList)):
        #home
        if newGameList[i][0] in avgStats:
            newGameList[i][2] = np.array(avgStats[newGameList[i][0]])/count
            avgStats[newGameList[i][0]] = np.sum([avgStats[newGameList[i][0]], newGameList[i][2]], axis=0)  #total stats + indivudal game stats
        else:
            avgStats[newGameList[i][0]] = newGameList[i][2]
            delList.append(i)

        #away
        if newGameList[i][3] in avgStats:
            newGameList[i][5] = np.array(avgStats[newGameList[i][3]])/count
            avgStats[newGameList[i][3]] = np.sum([avgStats[newGameList[i][3]], newGameList[i][5]], axis=0)
        else:
            avgStats[newGameList[i][3]] = newGameList[i][5]
            delList.append(i)
        
        count += 1
    
    for i in delList: del(newGameList[i])
    return newGameList
    
def normalizeStats(homeTeamStats, visitingTeamStats):
    # take the stats and zscore them, I do a number of transformations on the array because of how the zScore input contstraints 
    zScoredStatsHome = np.empty((0, len(homeTeamStats)))
    zScoredStatsVisitor = np.empty((0, len(visitingTeamStats)))
    
    for x in range(len(homeTeamStats[0])):
        zScoredStatsHome = np.vstack((zScoredStatsHome, stats.zscore([i[x] for i in homeTeamStats])))
        zScoredStatsVisitor = np.vstack((zScoredStatsVisitor, stats.zscore([i[x] for i in visitingTeamStats])))
    
    homeTeamStatsTemp = np.empty((len(zScoredStatsHome[0]), 0))
    visitingTeamStatsTemp = np.empty((len(zScoredStatsVisitor[0]), 0))
    for i in zScoredStatsHome:
        homeTeamStatsTemp = np.hstack((homeTeamStatsTemp, [[x] for x in i]))

    for i in zScoredStatsVisitor:
        visitingTeamStatsTemp = np.hstack((visitingTeamStatsTemp, [[x] for x in i]))
    
    return homeTeamStatsTemp, visitingTeamStatsTemp

Extract previously decided upon statistics (see accompanying paper) and games for processesing.

In [23]:
gameStats2016 = GameStats(6, 10, [50, 53, 54, 58, 63, 65, 68, 73, 75], 3, 9, [22, 25, 26, 30, 35, 37, 40, 45, 47])
gameStats2015 = GameStats(6, 10, [50, 53, 54, 58, 63, 65, 68, 73, 75], 3, 9, [22, 25, 26, 30, 35, 37, 40, 45, 47], file="baseball2015.txt")
gameList = gameStats2015.getGameList() # get the list of games played in 2015
gameList2016 = gameStats2016.getGameList() # get the list of games played in 2016

Process the statistics so that they are in a form better to run through the nural network.

In [30]:
# Attempt to calculate the stats of the current game by taking the average of all previously played games.
averagedGameList = previousAverage(gameList)

# Split the game information into multiple lists to make processing easier in the future.
homeTeamName, homeTeamScore, homeTeamStats, visitingTeamName, visitingTeamScore, visitingTeamStats = splitGameList(averagedGameList)

# Normalize the stats, by taking the z-score of each type of stat to ensure that one type of stat doesn't dominate the others.
normalizedHomeTeamStats, normalizedVisitingTeamStats = normalizeStats(homeTeamStats, visitingTeamStats)

# Repeat for 2016 games.
averagedGameList2016 = previousAverage(gameList2016)
homeTeamName2016, homeTeamScore2016, homeTeamStats2016, visitingTeamName2016, visitingTeamScore2016, visitingTeamStats2016 = splitGameList(averagedGameList2016)
normalizedHomeTeamStats2016, normalizedVisitingTeamStats2016 = normalizeStats(homeTeamStats2016, visitingTeamStats2016)

TypeError: ufunc 'true_divide' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [31]:
from hyperopt import hp, fmin, tpe
import math

def runModel(width, depth, dropOut, learningRate, HTN=homeTeamName, VTN=visitingTeamName, HTS=normalizedHomeTeamStats, VTS=normalizedVisitingTeamStats, GL=gameList):
    tf.reset_default_graph()

    # the input layer needs to have the same dimensions as our input (in this case the teams)
    homeTeamNameInput = tflearn.input_data(shape=[None, len(HTN[0])], name='nameInput1')
    homeTeamStatsInput = tflearn.input_data(shape=[None, len(HTS[0])], name='statsInput1')
    visitingTeamNameInput = tflearn.input_data(shape=[None, len(VTN[0])], name='nameInput2')
    visitingTeamStatsInput = tflearn.input_data(shape=[None, len(VTS[0])], name='statsInput2')

    nameProcess1 = tflearn.fully_connected(homeTeamNameInput, 4)
    nameProcess2 = tflearn.fully_connected(visitingTeamNameInput, 4)
    net = tflearn.layers.merge_ops.merge([nameProcess1, homeTeamStatsInput, nameProcess2, visitingTeamStatsInput], 'concat', axis=1)
    # next we have the hidden layer it is the feature matrix the size is arbitrary.
    for _ in range(depth):
        net = tflearn.fully_connected(net, width)
    
    net = tflearn.layers.normalization.batch_normalization(net) # normalize the output of the layers

    # The output layer
    net = tflearn.fully_connected(net, 2, activation="RELU") # apply relu to gaurentee no negative output
    net = tflearn.regression(net, name='target', learning_rate=learningRate)

    # take only the stats for each team and put them into an array
    NNOutput = [[i[1], i[4]] for i in GL]
    NNOutput = np.array(NNOutput)
    
    # Define model
    model = tflearn.DNN(net)
    # Start training (apply gradient descent algorithm)
    model.fit({'nameInput1':HTN, 'statsInput1':HTS, 'nameInput2':VTN, 'statsInput2':VTS}, NNOutput, validation_set=0.1, n_epoch=20, show_metric=True)
    return model, [model.predict([[HTN[i]], [HTS[i]], [VTN[i]], [VTS[i]]]) for i in range(len(HTS))]
    
def objective(args):
    width = args[0]
    depth = args[1]
    dropOut = 0
    learningRate = args[2]
    
    file = open("bayesTest.txt", "a+") 
    
    NNOutput = [[i[1], i[4]] for i in gameList]
    NNOutput = np.array(NNOutput)
    
    # Calculate the error with the current hyperparamters.
    predict = runModel(width, depth, dropOut, learningRate)
    error = np.sum(([(predict[1][i][0][0]-NNOutput[i][0])**2+(predict[1][i][0][1]-NNOutput[i][1])**2 for i in range(len(NNOutput))]))
    
    if(math.isnan(error) or error == 144414.0): # stop exploadig or disapering gradient from causing problems
        error = 10*(10**50)
    
    file.write(str(args))
    file.write("\t")
    file.write(str(error))
    file.write("\n---------------------------\n")
    file.close()
    
    return error

Determine appropriate hyperparamers of the model by using a bayesian optimizer. (Note: this takes about 25 minutes to run)

In [ ]:
open("bayesTest.txt", 'w').close() # Wipe the log file.

# Determine the best hyperparameters to use with the nural network.
space = [hp.choice('width', range(4, 30)), hp.choice('depth', range(3, 20)), hp.uniform('learningRate', 0, 0.05)]

# minimize the error on the objective function using tpe.
best = fmin(objective, space=space, algo=tpe.suggest, max_evals=50)
print(best)

In [58]:
print(np.array(runModel(best['width'], best['depth'], 0, best['learningRate'])))

Training Step: 699  | total loss: 5.45078 | time: 0.458s
| Adam | epoch: 020 | loss: 5.45078 - acc: 0.9258 -- iter: 2176/2185
Training Step: 700  | total loss: 5.42061 | time: 1.475s
| Adam | epoch: 020 | loss: 5.42061 - acc: 0.9270 | val_loss: 5.19843 - val_acc: 0.9630 -- iter: 2185/2185
--
[[[  9.834    8.9713]]

 [[  8.6805  15.5884]]

 [[ 11.3733   3.626 ]]

 ..., 
 [[ 20.9969   3.0485]]

 [[ 27.9909  13.0979]]

 [[ 29.7177  24.3521]]]


The following hyperparameters were previously chosen by the above algoirthm, it can be seen that when the averaged data is run through this model, it yields an accuracy of around 50%. While this doesn't indicate how far off the final output will be as this is only the relative error, it gives a good indication that something has gone wrong. (Note: this takes about 30 seconds to run)

In [258]:
model, data = runModel(20, 3, 0.812295321409586, 0.01811675281641176, GL=averagedGameList)

Training Step: 679  | total loss: 5.83258 | time: 0.383s
| Adam | epoch: 020 | loss: 5.83258 - acc: 0.5897 -- iter: 2112/2159
Training Step: 680  | total loss: 5.81964 | time: 1.398s
| Adam | epoch: 020 | loss: 5.81964 - acc: 0.6011 | val_loss: 6.03130 - val_acc: 0.5417 -- iter: 2159/2159
--


To test the actual accuracy of the model, the predicted output is taken and rounded (because scores are integer values).

In [276]:
roundedData = [[round(i[0][0]), round(i[0][1])] for i in data]
print(roundedData)

[[4, 0], [5, 0], [4, 2], [4, 2], [4, 2], [3, 3], [3, 2], [3, 2], [3, 3], [3, 3], [3, 2], [3, 3], [3, 2], [3, 3], [2, 3], [3, 3], [3, 3], [3, 3], [2, 3], [3, 3], [3, 3], [3, 3], [3, 3], [3, 3], [2, 3], [2, 3], [2, 3], [3, 3], [2, 3], [2, 3], [2, 3], [3, 3], [3, 3], [2, 3], [3, 3], [3, 3], [3, 3], [3, 3], [3, 3], [2, 3], [2, 3], [2, 3], [3, 3], [2, 3], [2, 3], [2, 3], [3, 3], [3, 3], [2, 3], [3, 3], [3, 3], [3, 3], [3, 3], [3, 3], [2, 3], [2, 3], [2, 3], [3, 3], [2, 3], [2, 3], [2, 3], [3, 3], [3, 2], [3, 2], [3, 3], [3, 3], [3, 2], [2, 3], [3, 2], [3, 2], [3, 2], [3, 2], [2, 3], [3, 2], [3, 2], [3, 3], [3, 2], [3, 3], [3, 2], [3, 3], [3, 2], [2, 3], [3, 2], [3, 2], [3, 2], [3, 2], [2, 3], [3, 2], [3, 3], [3, 2], [3, 3], [3, 2], [3, 3], [3, 3], [3, 2], [2, 3], [3, 2], [3, 2], [3, 2], [3, 2], [2, 3], [3, 2], [3, 2], [3, 3], [3, 2], [3, 2], [3, 3], [3, 2], [3, 2], [3, 2], [2, 3], [3, 2], [3, 2], [3, 3], [3, 3], [3, 3], [3, 1], [3, 3], [3, 2], [3, 2], [3, 2], [3, 3], [3, 2], [3, 2], [3, 2],

In [278]:
NNOutput = [[round(i[1]), round(i[4])] for i in averagedGameList]
print(NNOutput)

[[2, 0], [8, 0], [4, 5], [1, 2], [1, 3], [5, 6], [2, 12], [0, 2], [10, 0], [2, 5], [7, 4], [4, 2], [1, 5], [1, 10], [0, 1], [3, 6], [2, 4], [5, 12], [0, 6], [4, 8], [5, 6], [12, 0], [1, 5], [4, 3], [5, 3], [5, 4], [5, 1], [10, 9], [2, 6], [4, 1], [1, 0], [4, 6], [7, 1], [5, 4], [6, 9], [4, 8], [4, 5], [6, 2], [6, 0], [5, 3], [1, 4], [5, 9], [0, 2], [6, 0], [3, 2], [10, 2], [2, 9], [7, 10], [6, 2], [5, 8], [14, 4], [7, 8], [4, 6], [4, 7], [3, 4], [5, 7], [5, 6], [5, 8], [2, 10], [3, 4], [6, 4], [5, 6], [9, 4], [1, 8], [3, 12], [3, 6], [1, 2], [3, 2], [7, 6], [6, 5], [2, 0], [5, 4], [4, 8], [0, 2], [4, 5], [4, 3], [8, 7], [1, 4], [0, 4], [8, 2], [2, 3], [2, 8], [2, 3], [6, 5], [6, 5], [0, 2], [5, 1], [1, 4], [7, 5], [5, 10], [4, 2], [6, 1], [3, 1], [2, 10], [12, 7], [2, 6], [5, 0], [5, 2], [6, 1], [0, 1], [3, 2], [2, 4], [4, 2], [8, 5], [2, 4], [7, 5], [6, 7], [4, 0], [5, 2], [3, 2], [2, 1], [3, 6], [6, 4], [3, 2], [1, 3], [4, 5], [7, 8], [4, 5], [7, 3], [4, 1], [6, 3], [0, 9], [6, 1], [

Then the predicted output is subtracted from the actual output and the results are added to give a general idea of how far off the predictions are. The average absolute error shows that the scores are only off by and average of 2.5, although the maximum socre difference does indicate that there is a high varience on that number. Having an error of only 2.5 isn't bad, but only about 40% of the wins were correctly predicted which means that while the scores may have been within the same ball park they did not correlate very well to the actual events of the game.

In [309]:
print("Averge Absolute Error:", sum(abs(np.array(NNOutput)-np.array(roundedData)))/len(NNOutput))
print("Maximum Score Difference:", [max([abs(NNOutput[i][0]-roundedData[i][0]) for i in range(len(NNOutput))]), max([abs(NNOutput[i][1]-roundedData[i][1]) for i in range(len(NNOutput))])])

print("Percent Wins Correct:", sum([True if (NNOutput[i][0]-NNOutput[i][1])*(roundedData[i][0]-roundedData[i][1]) > 0 else False for i in range(len(NNOutput))])/len(NNOutput)*100)

Averge Absolute Error: [ 2.4748  2.4981]
Maximum Score Difference: [19, 18]
Percent Wins Correct: 38.89120466861192


Running the same analysis on the 2016 season.

In [279]:
data2016 = [model.predict([[homeTeamName2016[i]], [normalizedHomeTeamStats2016[i]], [visitingTeamName2016[i]], [normalizedVisitingTeamStats2016[i]]]) for i in range(len(homeTeamStats2016))]

In [281]:
NNOutput2016 = [[round(i[1]), round(i[4])] for i in averagedGameList2016]
print(NNOutput2016)

[[4, 1], [3, 4], [5, 10], [3, 12], [2, 6], [4, 5], [3, 2], [7, 8], [0, 3], [16, 6], [5, 9], [3, 2], [5, 1], [0, 7], [8, 5], [1, 6], [12, 6], [4, 6], [3, 7], [6, 1], [1, 7], [2, 3], [7, 8], [3, 2], [4, 7], [5, 6], [6, 13], [6, 4], [7, 2], [3, 2], [1, 4], [7, 3], [4, 8], [7, 0], [1, 6], [4, 8], [2, 4], [2, 12], [5, 1], [3, 16], [4, 6], [0, 1], [2, 3], [3, 1], [5, 3], [4, 3], [1, 2], [3, 0], [3, 7], [7, 12], [2, 1], [6, 3], [3, 2], [2, 5], [9, 6], [4, 2], [7, 9], [4, 7], [8, 2], [1, 4], [1, 4], [3, 7], [5, 3], [3, 10], [3, 4], [10, 1], [6, 4], [5, 9], [8, 2], [2, 3], [4, 5], [0, 8], [5, 1], [2, 3], [2, 7], [2, 4], [1, 2], [3, 0], [2, 1], [4, 2], [2, 4], [0, 3], [1, 5], [4, 2], [1, 4], [7, 2], [9, 2], [10, 6], [3, 1], [2, 1], [2, 1], [3, 7], [4, 6], [3, 0], [2, 6], [1, 3], [0, 6], [6, 3], [4, 2], [8, 1], [11, 6], [5, 2], [3, 0], [4, 7], [7, 0], [6, 2], [5, 3], [5, 6], [1, 0], [5, 4], [1, 7], [2, 4], [0, 1], [5, 11], [1, 6], [7, 3], [3, 6], [1, 9], [4, 8], [2, 3], [14, 3], [4, 2], [7, 5], [

In [282]:
roundedData2016 = [[round(i[0][0]), round(i[0][1])] for i in data2016]
print(roundedData2016)

[[3, 2], [2, 3], [3, 2], [3, 2], [3, 2], [3, 3], [2, 3], [3, 2], [2, 3], [3, 3], [2, 3], [2, 3], [3, 3], [2, 3], [3, 3], [3, 3], [2, 3], [3, 3], [3, 2], [3, 3], [3, 2], [3, 3], [3, 2], [2, 3], [2, 3], [2, 3], [3, 3], [3, 2], [2, 3], [2, 3], [3, 2], [3, 2], [3, 2], [2, 3], [3, 3], [3, 2], [3, 3], [2, 3], [2, 3], [2, 3], [3, 2], [2, 3], [2, 3], [3, 2], [3, 3], [3, 3], [3, 3], [3, 2], [2, 3], [3, 3], [2, 3], [2, 3], [3, 2], [2, 3], [2, 3], [3, 3], [3, 3], [2, 3], [3, 3], [3, 2], [2, 3], [3, 3], [3, 2], [3, 3], [3, 2], [3, 2], [3, 3], [3, 3], [2, 3], [2, 3], [2, 3], [3, 2], [3, 3], [3, 2], [2, 3], [3, 2], [3, 3], [3, 2], [3, 3], [3, 2], [3, 3], [3, 2], [3, 3], [3, 3], [3, 3], [3, 2], [3, 2], [2, 3], [3, 2], [3, 3], [3, 2], [3, 2], [3, 2], [3, 3], [2, 3], [3, 2], [3, 3], [2, 3], [3, 2], [3, 2], [2, 3], [3, 3], [3, 2], [3, 2], [3, 3], [3, 3], [3, 2], [3, 2], [3, 2], [3, 2], [2, 3], [2, 3], [3, 3], [2, 3], [3, 2], [3, 2], [3, 2], [3, 2], [3, 2], [3, 3], [3, 2], [3, 2], [3, 3], [3, 2], [3, 2],

As one would expect when not run on the training dataset the model did even worse (although the maximum score difference was smaller).

In [310]:
print("Averge Absolute Error:", sum(abs(np.array(NNOutput2016)-np.array(roundedData2016)))/len(NNOutput2016))
print("Maximum Score Difference:", [max([abs(NNOutput2016[i][0]-roundedData2016[i][0]) for i in range(len(NNOutput2016))]), max([abs(NNOutput2016[i][1]-roundedData2016[i][1]) for i in range(len(NNOutput2016))])])

print("Percent Wins Correct:", sum([True if (NNOutput2016[i][0]-NNOutput2016[i][1])*(roundedData2016[i][0]-roundedData2016[i][1]) > 0 else False for i in range(len(NNOutput2016))])/len(NNOutput2016)*100)

Averge Absolute Error: [ 2.6818  2.666 ]
Maximum Score Difference: [15, 18]
Percent Wins Correct: 32.610508757297744


It is clear that the model while getting numbers that are close to the actual scores is not very good. But the question remains why? Analysis of the statistics used (see the paper) shows that there does seem to be some correlation between these stats and the results of the game. Therefore it seems reasonable that it was the method by which the statistics were calculated that was the problem. This is reasonable because when the teams were averaged what came out was not how well each team did in each matchup but where they ranked. In addition when looking at the scores it seems like they converge on a couple of values which is expected when results are being averaged, but does not model real world behaviour very well. 

Testing to see if this theory is correct simply involves assuming that the stats of each game are known beforehand. If these "perfect" stats are run through the model and results improve then the poor results of the first attempt can be partially attributed to the method bywhich the stats were initially calculated and not by model.

The reuslts are gathered and processed in the same as before, but this time they are not averaged. 

In [7]:
gameStats2016 = GameStats(6, 10, [50, 53, 54, 58, 63, 65, 68, 73, 75], 3, 9, [22, 25, 26, 30, 35, 37, 40, 45, 47])
gameStats2015 = GameStats(6, 10, [50, 53, 54, 58, 63, 65, 68, 73, 75], 3, 9, [22, 25, 26, 30, 35, 37, 40, 45, 47], file="baseball2015.txt")

gameList = gameStats2015.getGameList() # get the list of games played in 2015
gameList2016 = gameStats2016.getGameList() # get the list of games played in 2016

# Split the game information into multiple lists to make processing easier in the future.
homeTeamName, homeTeamScore, homeTeamStats, visitingTeamName, visitingTeamScore, visitingTeamStats = splitGameList(gameList)

# Normalize the stats, by taking the z-score of each type of stat to ensure that one type of stat doesn't dominate the others.
normalizedHomeTeamStats, normalizedVisitingTeamStats = normalizeStats(homeTeamStats, visitingTeamStats)

# Repeat for 2016 games.
homeTeamName2016, homeTeamScore2016, homeTeamStats2016, visitingTeamName2016, visitingTeamScore2016, visitingTeamStats2016 = splitGameList(gameList2016)
normalizedHomeTeamStats2016, normalizedVisitingTeamStats2016 = normalizeStats(homeTeamStats2016, visitingTeamStats2016)

The model is rerun and the results are already looking a lot better. (Note: this takes about 30 seconds to run)

In [10]:
model, newData = runModel(20, 3, 0.812295321409586, 0.01811675281641176)

Training Step: 699  | total loss: 4.88555 | time: 0.365s
| Adam | epoch: 020 | loss: 4.88555 - acc: 0.9675 -- iter: 2176/2186
Training Step: 700  | total loss: 4.87028 | time: 1.382s
| Adam | epoch: 020 | loss: 4.87028 - acc: 0.9661 | val_loss: 5.00436 - val_acc: 0.9794 -- iter: 2186/2186
--


In [20]:
roundedData = [[round(i[0][0]), round(i[0][1])] for i in newData]
print(roundedData)

[[0, 3], [4, 1], [2, 0], [8, 2], [1, 4], [7, 1], [4, 2], [2, 5], [3, 4], [4, 2], [5, 3], [1, 2], [0, 9], [0, 6], [1, 2], [1, 3], [0, 2], [4, 5], [6, 5], [3, 5], [1, 10], [1, 5], [9, 1], [0, 2], [6, 5], [3, 3], [9, 1], [3, 4], [2, 0], [1, 4], [2, 0], [4, 3], [6, 4], [0, 2], [3, 5], [4, 2], [2, 1], [6, 1], [1, 5], [4, 1], [3, 5], [1, 8], [2, 2], [1, 6], [1, 1], [3, 5], [2, 4], [4, 10], [0, 4], [3, 6], [5, 5], [11, 1], [1, 5], [4, 3], [4, 3], [4, 3], [5, 1], [7, 8], [2, 5], [3, 1], [1, 1], [3, 5], [6, 2], [4, 4], [4, 7], [3, 6], [4, 4], [4, 2], [5, 1], [4, 2], [2, 4], [4, 8], [0, 2], [5, 1], [3, 2], [9, 3], [1, 7], [5, 8], [6, 2], [4, 7], [12, 4], [6, 6], [3, 6], [4, 6], [2, 4], [4, 6], [5, 6], [4, 7], [2, 6], [3, 3], [5, 4], [4, 5], [7, 4], [1, 7], [2, 9], [3, 5], [1, 2], [2, 2], [6, 6], [5, 5], [2, 1], [4, 4], [3, 5], [0, 2], [3, 4], [4, 2], [5, 6], [1, 3], [0, 4], [7, 2], [2, 3], [2, 7], [2, 3], [5, 4], [5, 5], [0, 2], [4, 2], [1, 3], [6, 4], [3, 8], [3, 2], [5, 2], [3, 1], [1, 8], [10

In [21]:
NNOutput = [[round(i[1]), round(i[4])] for i in gameList]
print(NNOutput)

[[0, 3], [4, 0], [2, 0], [10, 1], [1, 6], [8, 0], [4, 1], [2, 6], [4, 5], [5, 2], [6, 3], [1, 2], [0, 10], [0, 8], [1, 3], [1, 3], [0, 2], [5, 6], [7, 6], [3, 7], [2, 12], [2, 5], [11, 0], [0, 2], [7, 5], [4, 3], [10, 0], [3, 5], [2, 0], [2, 5], [2, 0], [5, 4], [7, 4], [0, 2], [4, 5], [4, 2], [2, 1], [7, 1], [1, 5], [4, 1], [3, 6], [1, 10], [3, 2], [2, 6], [0, 1], [3, 6], [2, 4], [5, 12], [0, 6], [4, 8], [5, 6], [12, 0], [1, 5], [4, 3], [5, 3], [5, 4], [5, 1], [10, 9], [2, 6], [4, 1], [1, 0], [4, 6], [7, 1], [5, 4], [6, 9], [4, 8], [4, 5], [6, 2], [6, 0], [5, 3], [1, 4], [5, 9], [0, 2], [6, 0], [3, 2], [10, 2], [2, 9], [7, 10], [6, 2], [5, 8], [14, 4], [7, 8], [4, 6], [4, 7], [3, 4], [5, 7], [5, 6], [5, 8], [2, 10], [3, 4], [6, 4], [5, 6], [9, 4], [1, 8], [3, 12], [3, 6], [1, 2], [3, 2], [7, 6], [6, 5], [2, 0], [5, 4], [4, 8], [0, 2], [4, 5], [4, 3], [8, 7], [1, 4], [0, 4], [8, 2], [2, 3], [2, 8], [2, 3], [6, 5], [6, 5], [0, 2], [5, 1], [1, 4], [7, 5], [5, 10], [4, 2], [6, 1], [3, 1], 

In [22]:
print("Averge Absolute Error:", sum(abs(np.array(NNOutput)-np.array(roundedData)))/len(NNOutput))
print("Maximum Score Difference:", [max([abs(NNOutput[i][0]-roundedData[i][0]) for i in range(len(NNOutput))]), max([abs(NNOutput[i][1]-roundedData[i][1]) for i in range(len(NNOutput))])])

print("Percent Wins Correct:", sum([True if (NNOutput[i][0]-NNOutput[i][1])*(roundedData[i][0]-roundedData[i][1]) > 0 else False for i in range(len(NNOutput))])/len(NNOutput)*100)

Averge Absolute Error: [ 0.68587896  0.71716756]
Maximum Score Difference: [12, 16]
Percent Wins Correct: 88.88431453272952


In [15]:
data2016 = [model.predict([[homeTeamName2016[i]], [normalizedHomeTeamStats2016[i]], [visitingTeamName2016[i]], [normalizedVisitingTeamStats2016[i]]]) for i in range(len(homeTeamStats2016))]

In [16]:
roundedData2016 = [[round(i[0][0]), round(i[0][1])] for i in data2016]
print(roundedData2016)

[[3, 2], [3, 4], [3, 1], [0, 7], [3, 2], [3, 2], [2, 4], [2, 2], [4, 8], [3, 4], [5, 2], [2, 10], [0, 11], [1, 5], [1, 5], [0, 1], [2, 3], [3, 4], [3, 1], [1, 8], [9, 6], [6, 8], [0, 2], [5, 4], [0, 3], [3, 2], [6, 5], [13, 6], [2, 1], [4, 3], [3, 8], [2, 4], [1, 3], [3, 2], [2, 6], [3, 3], [5, 1], [0, 5], [4, 3], [3, 2], [7, 4], [1, 5], [4, 10], [9, 5], [10, 6], [4, 5], [2, 5], [5, 2], [1, 5], [4, 1], [3, 2], [2, 3], [5, 7], [3, 2], [3, 5], [4, 5], [4, 11], [5, 4], [6, 2], [2, 2], [1, 3], [7, 3], [3, 6], [6, 0], [0, 5], [3, 6], [1, 4], [2, 8], [5, 2], [1, 13], [3, 6], [0, 1], [2, 3], [3, 1], [4, 3], [3, 3], [0, 2], [2, 0], [2, 6], [5, 9], [2, 1], [5, 3], [3, 2], [1, 4], [7, 5], [4, 2], [6, 7], [3, 6], [7, 3], [1, 2], [1, 3], [2, 6], [4, 3], [2, 8], [2, 3], [9, 1], [5, 4], [3, 7], [7, 3], [2, 3], [4, 4], [0, 6], [4, 1], [2, 2], [1, 6], [2, 4], [1, 2], [3, 0], [2, 1], [3, 2], [2, 3], [0, 2], [1, 4], [4, 2], [1, 3], [5, 2], [7, 2], [8, 6], [3, 1], [2, 1], [3, 1], [2, 5], [3, 5], [3, 0], 

In [18]:
NNOutput2016 = [[round(i[1]), round(i[4])] for i in gameList2016]
print(NNOutput2016)

[[4, 3], [3, 5], [4, 1], [0, 9], [3, 2], [3, 4], [3, 5], [3, 2], [5, 10], [3, 4], [6, 2], [3, 12], [0, 15], [1, 6], [2, 6], [0, 2], [3, 5], [4, 5], [3, 2], [2, 10], [11, 6], [7, 8], [1, 2], [6, 5], [0, 3], [4, 2], [7, 6], [16, 6], [2, 1], [5, 3], [5, 9], [3, 4], [1, 3], [3, 2], [3, 7], [4, 3], [5, 1], [0, 7], [4, 3], [4, 2], [8, 5], [1, 6], [6, 14], [10, 6], [12, 6], [4, 6], [3, 7], [6, 1], [1, 7], [4, 0], [4, 3], [2, 3], [7, 8], [3, 2], [4, 7], [5, 6], [6, 13], [6, 4], [7, 2], [3, 2], [1, 4], [7, 3], [4, 8], [7, 0], [1, 6], [4, 8], [2, 4], [2, 12], [5, 1], [3, 16], [4, 6], [0, 1], [2, 3], [3, 1], [5, 3], [4, 3], [1, 2], [3, 0], [3, 7], [7, 12], [2, 1], [6, 3], [3, 2], [2, 5], [9, 6], [4, 2], [7, 9], [4, 7], [8, 2], [1, 4], [1, 4], [3, 7], [5, 3], [3, 10], [3, 4], [10, 1], [6, 4], [5, 9], [8, 2], [2, 3], [4, 5], [0, 8], [5, 1], [2, 3], [2, 7], [2, 4], [1, 2], [3, 0], [2, 1], [4, 2], [2, 4], [0, 3], [1, 5], [4, 2], [1, 4], [7, 2], [9, 2], [10, 6], [3, 1], [2, 1], [2, 1], [3, 7], [4, 6],

The results when using the "perfect" data, are far better than before indicating that the problem was likely with the method bywhich we calculated the data in the first place.

In [19]:
print("Averge Absolute Error:", sum(abs(np.array(NNOutput2016)-np.array(roundedData2016)))/len(NNOutput2016))
print("Maximum Score Difference:", [max([abs(NNOutput2016[i][0]-roundedData2016[i][0]) for i in range(len(NNOutput2016))]), max([abs(NNOutput2016[i][1]-roundedData2016[i][1]) for i in range(len(NNOutput2016))])])

print("Percent Wins Correct:", sum([True if (NNOutput2016[i][0]-NNOutput2016[i][1])*(roundedData2016[i][0]-roundedData2016[i][1]) > 0 else False for i in range(len(NNOutput2016))])/len(NNOutput2016)*100)

Averge Absolute Error: [ 0.83031301  0.73146623]
Maximum Score Difference: [8, 15]
Percent Wins Correct: 90.19769357495882
